## Transcrire un algorithme (Grover)

### Registres

In [1]:
from qiskit import QuantumRegister, ClassicalRegister

valeurs = QuantumRegister(3, 'valeurs')
mesures = ClassicalRegister(valeurs.size, 'mesures')

### Oracle

In [ ]:
from qiskit import QuantumCircuit

oracle = QuantumCircuit(valeurs.size)
oracle.barrier(range(oracle.num_qubits))
oracle.x(1)
oracle.ccz(0, 1, 2)
oracle.x(1)
oracle.barrier(range(oracle.num_qubits))
oracle.draw()

### Diffuseur

In [ ]:
diffuseur = QuantumCircuit(valeurs.size)
diffuseur.h(range(diffuseur.num_qubits))
diffuseur.x(range(diffuseur.num_qubits))
# diffuseur.barrier(range(diffuseur.num_qubits-1))
diffuseur.h(diffuseur.num_qubits-1)
diffuseur.ccx(0, 1, 2)
diffuseur.h(diffuseur.num_qubits-1)
# diffuseur.barrier(range(diffuseur.num_qubits-1))
diffuseur.x(range(diffuseur.num_qubits))
diffuseur.h(range(diffuseur.num_qubits))
diffuseur.draw()

### Circuit

#### Amorce
Portes d'Hadamard + Oracle

In [ ]:
circuit = QuantumCircuit(valeurs)
circuit.h(range(circuit.num_qubits))
circuit = circuit.compose(oracle)
circuit.draw()

#### Vérification de l'oracle
Simulation partielle

In [6]:
from qiskit import transpile, BasicAer

simulator = BasicAer.get_backend('statevector_simulator')
result = simulator.run(transpile(circuit, simulator), shots=256).result()
states = result.get_statevector(circuit)

Phases

In [ ]:
from numpy import arctan2, degrees, round

[round(degrees(arctan2(state.imag, state.real))) for state in states]

Graphique

In [ ]:
from qiskit.visualization import plot_state_qsphere

plot_state_qsphere(states, show_state_phases=True)

#### Ajout du diffuseur

In [ ]:
circuit.compose(diffuseur, inplace=True)
circuit.draw()

#### Répétition + Mesure

In [ ]:
circuit = circuit \
    .compose(oracle) \
    .compose(diffuseur)

circuit.add_register(mesures)
circuit.measure_all(add_bits=False)
circuit.draw()

### Simulation

In [ ]:
from qiskit import transpile, BasicAer
from qiskit.visualization import plot_histogram

simulator = BasicAer.get_backend('qasm_simulator')
compiled = transpile(circuit, simulator)
job = simulator.run(compiled, shots=1000)
result = job.result()

counts = result.get_counts(compiled).int_outcomes()
plot_histogram(counts)